In [41]:
import os
import sys
import time
import datetime
import socket
import yaml
import shutil

import pandas as pd
import numpy as np

from sklearn.externals import joblib 

from sklearn.metrics import accuracy_score, log_loss, f1_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [42]:
folderFeature = 'feature1'
feaPaths = ['../xprmt/'+folderFeature+'/extract-slbp-DataPH2-mlbox-20181116-021021/DataPH2',
           '../xprmt/'+folderFeature+'/extract-haralick-DataPH2-mlbox-20181116-021704/DataPH2',
           '../xprmt/'+folderFeature+'/extract-wtio-DataPH2-mlbox-20181116-021739/DataPH2',
          ]

featureList = []
labelList = []
for feaPath in feaPaths:
    print(feaPath)
    xListL = []
    yListL = []
    yLi = os.listdir(feaPath)
    sumdat = 0
    for i in yLi:
        file = os.listdir(feaPath+ "/" + i)
        cnt = 0
        for j in file:
            if j.endswith('.pkl'):
                #print(j)
                x = joblib.load(feaPath + "/" + i + "/" +j)
                xListL.append(x)
                yListL.append(i)
                cnt += 1
        print(str(i) + " " + str(cnt))
        sumdat += cnt
    
    print("semuanya "+ str(sumdat))
    featureList.append(xListL)
    labelList.append(yListL)

xList = np.concatenate((featureList[0], featureList[1]), axis=1)
for idx in range(len(featureList)-2):
    xList = np.concatenate((xList, featureList[idx+2]), axis=1)

xList1 = xList
yList1 = labelList[0]

../xprmt/feature1/extract-slbp-DataPH2-mlbox-20181116-021021/DataPH2
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240
../xprmt/feature1/extract-haralick-DataPH2-mlbox-20181116-021704/DataPH2
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240
../xprmt/feature1/extract-wtio-DataPH2-mlbox-20181116-021739/DataPH2
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240


In [43]:
folderFeature = 'feature1'
feaPaths = ['../xprmt/'+folderFeature+'/extract-slbp-DataPH2_lesion-mlbox-20181116-163545/DataPH2_lesion',
           '../xprmt/'+folderFeature+'/extract-haralick-DataPH2_lesion-mlbox-20181116-184443/DataPH2_lesion',
           '../xprmt/'+folderFeature+'/extract-wtio-DataPH2_lesion-mlbox-20181116-151859/DataPH2_lesion',
          ]

featureList = []
labelList = []
for feaPath in feaPaths:
    print(feaPath)
    xListL = []
    yListL = []
    yLi = os.listdir(feaPath)
    sumdat = 0
    for i in yLi:
        file = os.listdir(feaPath+ "/" + i)
        cnt = 0
        for j in file:
            if j.endswith('.pkl'):
                #print(j)
                x = joblib.load(feaPath + "/" + i + "/" +j)
                xListL.append(x)
                yListL.append(i)
                cnt += 1
        print(str(i) + " " + str(cnt))
        sumdat += cnt
    
    print("semuanya "+ str(sumdat))
    featureList.append(xListL)
    labelList.append(yListL)

xList = np.concatenate((featureList[0], featureList[1]), axis=1)
for idx in range(len(featureList)-2):
    xList = np.concatenate((xList, featureList[idx+2]), axis=1)

xList2 = xList
yList2 = labelList[0]

../xprmt/feature1/extract-slbp-DataPH2_lesion-mlbox-20181116-163545/DataPH2_lesion
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240
../xprmt/feature1/extract-haralick-DataPH2_lesion-mlbox-20181116-184443/DataPH2_lesion
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240
../xprmt/feature1/extract-wtio-DataPH2_lesion-mlbox-20181116-151859/DataPH2_lesion
Melanoma 80
Atypical Nevus 80
Common Nevus 80
semuanya 240


In [44]:
# feaPath = '../xprmt/feature1/extract-slbp-DataPH2_lesion-mlbox-20181116-163545/DataPH2_lesion'

# xListL = []
# yListL = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListL.append(x)
#             yListL.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))

In [45]:
# feaPath = '../xprmt/feature1/extract-haralick-DataPH2_lesion-mlbox-20181116-184443/DataPH2_lesion'

# xListH = []
# yListH = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListH.append(x)
#             yListH.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))

In [46]:
# feaPath = '../xprmt/feature1/extract-wtio-DataPH2_lesion-mlbox-20181116-151859/DataPH2_lesion'

# xListW = []
# yListW = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListW.append(x)
#             yListW.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))

In [47]:
# xListL = np.array(xListL)
# xListH = np.array(xListH)
# xListW = np.array(xListW)
# xList1 = np.concatenate((xListL, xListH, xListW), axis=1)
# #xList = np.concatenate((xListL, xListW), axis=1)
# print(yListL == yListH)
# print(yListH == yListW)
# yList1 = yListL

In [48]:
# feaPath = '../xprmt/feature1/extract-slbp-DataPH2-mlbox-20181116-021021/DataPH2'

# xListL = []
# yListL = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListL.append(x)
#             yListL.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))
# feaPath = '../xprmt/feature1/extract-haralick-DataPH2-mlbox-20181116-021704/DataPH2'

# xListH = []
# yListH = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListH.append(x)
#             yListH.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))
# feaPath = '../xprmt/feature1/extract-wtio-DataPH2-mlbox-20181116-021739/DataPH2'

# xListW = []
# yListW = []
# yLi = os.listdir(feaPath)
# sumdat = 0
# for i in yLi:
#     file = os.listdir(feaPath+ "/" + i)
#     cnt = 0
#     for j in file:
#         if j.endswith('.pkl'):
#             #print(j)
#             x = joblib.load(feaPath + "/" + i + "/" +j)
#             xListW.append(x)
#             yListW.append(i)
#             cnt += 1
#     print(str(i) + " " + str(cnt))
#     sumdat += cnt
# print("semuanya "+ str(sumdat))

# xListL = np.array(xListL)
# xListH = np.array(xListH)
# xListW = np.array(xListW)
# xList2 = np.concatenate((xListL, xListH, xListW), axis=1)
# #xList = np.concatenate((xListL, xListW), axis=1)
# print(yListL == yListH)
# print(yListH == yListW)
# yList2 = yListL

In [49]:
xList = np.concatenate((xList1, xList2), axis=0)
yList1.extend(yList2)
yList = yList1
print(len(yList))

480


In [50]:
xList = np.array(xList)
yList = np.array(yList)
print(xList.shape)
print(yList.shape)

(480, 2079)
(480,)


In [51]:
from sklearn.preprocessing import normalize
xList = normalize(xList, norm='l2', axis=0)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xList, yList, random_state = 42, test_size = 0.2)

In [53]:
print(X_train.shape)

(384, 2079)


In [54]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import sensitivity_specificity_support
clf = RandomForestClassifier(random_state=6)
clf.fit(X_train, y_train)
prediksi = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, prediksi))
print(classification_report(y_test, prediksi))
print(sensitivity_specificity_support(y_test, prediksi, average='macro'))

0.84375
                precision    recall  f1-score   support

Atypical Nevus       0.77      0.82      0.79        28
  Common Nevus       0.78      0.78      0.78        32
      Melanoma       0.97      0.92      0.94        36

     micro avg       0.84      0.84      0.84        96
     macro avg       0.84      0.84      0.84        96
  weighted avg       0.85      0.84      0.85        96

(0.839781746031746, 0.923672385620915, None)


In [55]:
clf1 = RandomForestClassifier(random_state=1)
clf2 = GradientBoostingClassifier()
clf3 = LinearDiscriminantAnalysis()
eclf1 = VotingClassifier(estimators=[('rf', clf1), ('gb', clf2), ('lda', clf3)], voting='hard')
score = cross_val_score(clf, xList, yList, cv=10)
print(score)
print(score.mean())

[0.89583333 0.875      0.95833333 0.85416667 0.91666667 0.91666667
 0.91666667 0.91666667 0.95833333 0.85416667]
0.90625


In [56]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
score = cross_val_score(clf, xList, yList, cv=10)
print(score)
print(score.mean())

[0.875      0.875      0.91666667 0.91666667 0.875      0.95833333
 0.875      0.91666667 0.89583333 0.9375    ]
0.9041666666666666


In [ ]:

classifiers = [
    KNeighborsClassifier(3),
    #SVC(kernel="rbf", C=0.025, probability=True),
    #NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
    ######################### perform 10 fold validation ######################
    kf = StratifiedKFold(n_splits=10)
    scorelist = []
    sensitivity = []
    specificity = []
    f1 = []
    for train_index, test_index in kf.split(xList, yList):
        clf.fit(xList[train_index], yList[train_index])
        p = clf.predict(xList[test_index])
        accval = accuracy_score(yList[test_index], p)
        scorelist.append(accval)
        sss = sensitivity_specificity_support(yList[test_index], p, average='macro')
        sensitivity.append(sss[0])
        specificity.append(sss[1])
        f1.append(f1_score(yList[test_index], p, average='macro') )

        
    print("MeanCVScore: {}".format(sum(scorelist)/len(scorelist)))
    print("10FoldCVScore: {}".format(scorelist))
    print("sensitivity: {}".format(sum(sensitivity)/len(sensitivity)))
    print("specificity: {}".format(sum(specificity)/len(specificity)))
    print("f1-score: {}".format(sum(f1)/len(f1)))

    #############################################################################
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 66.6667%
Log Loss: 2.8425617495213658
MeanCVScore: 0.69375
10FoldCVScore: [0.7291666666666666, 0.6666666666666666, 0.6875, 0.6666666666666666, 0.6875, 0.6875, 0.7083333333333334, 0.75, 0.6666666666666666, 0.6875]
sensitivity: 0.69375
specificity: 0.846875
f1-score: 0.6961262199962062
DecisionTreeClassifier
****Results****
Accuracy: 81.2500%
Log Loss: 6.476020574045754
MeanCVScore: 0.8583333333333334
10FoldCVScore: [0.8125, 0.9375, 0.8958333333333334, 0.75, 0.8958333333333334, 0.875, 0.875, 0.8541666666666666, 0.8125, 0.875]
sensitivity: 0.8583333333333334
specificity: 0.9291666666666666
f1-score: 0.8594917023125653
RandomForestClassifier
****Results****
Accuracy: 89.5833%
Log Loss: 0.3557792184208239
MeanCVScore: 0.875
10FoldCVScore: [0.8333333333333334, 0.8541666666666666, 0.875, 0.875, 0.8333333333333334, 0.875, 0.7916666666666666, 0.9791666666666666, 0.9583333333333334, 0.875]
sensitivity: 0.875
specificity: 0.9375
f1-score: 0.876460494